In [136]:
import pandas as pd
import re

# the top locations that make up targetPercent of the worlds population for the given year
# data taken from the given file (assumed to be the output of the reformated population data)
# populations are in thousands

fileName = "data/TotalPopulation_Medium.csv"
year = "2020"
targetPercent = 0.5  # 0 to 1

popDF = pd.read_csv(fileName)

# extract the totals per country from the data
yearData = popDF[["Location", "Group", year]]
yearTotals = yearData.loc[yearData["Group"] == "Total"][["Location", year]]  # a DF with the location and its total population for the year given
# print(yearTotals)

# extract the world total
worldTotal = yearTotals.loc[yearTotals["Location"] == "World"][year].item()  # get the total world population for the year as given by the data
# print(worldTotal)

# remove all non-country locations
#----------------------------------Start Melanie's edits ---------------------------------
#Remove bad characters and unwanted spaces

with open("country_list.csv") as country_list:
    countries_file = country_list.read()
    
countries = re.sub('[^a-zA-Z\s]', '', countries_file).split()
countriesDF = pd.DataFrame(data = {"countries": countries})
# print(countriesDF.countries)

#make a common dataframe that has only countries and removes non-countries, dropping na values
common = countriesDF.merge(yearTotals, how='inner', left_on=['countries'], right_on=['Location'])
common = common.dropna()
# print(common)
# common = common.drop(['Location'], axis=1)
# print(common)
#----------------------------------End Melanie's edits ---------------------------------


# find the percent of the world's total by location and add a column with that
# yearTotals["Percent"] = yearTotals[year]/worldTotal
common["Percent"] = common["2020"]/worldTotal

# sort by percent of world population
# yearTotals.sort_values(by=["Percent"], ascending=False, inplace=True)
common.sort_values(by=["Percent"], ascending=False, inplace=True)
# print("year totals: ",yearTotals)

# the locations whos population is greater than targetPercent
# topLocations = yearTotals.loc[yearTotals["Percent"] > targetPercent]
topLocations = common.loc[common["Percent"] > targetPercent]

# print("top locations: ",topLocations)

# finds the indexes of the locations that collectively make up targetPercent of the world's population
topIndexes = []
cumsum = 0
# for index, series in yearTotals.iterrows():
for index, series in common.iterrows():
    cumsum += series[year]
    if cumsum/worldTotal > targetPercent:
        # print(topIndexes)
        break
    topIndexes.append(index)

# create data frame using the indexes from above (top locations using the cumulative sum)
# topCumLocations = yearTotals.loc[topIndexes]
topCumLocations = common.loc[topIndexes]

print("top cumulative locations: \n",topCumLocations)


top cumulative locations: 
    countries Location         2020   Percent
28     China    China  1424548.266  0.182740
60     India    India  1383197.753  0.177436
